In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

%config Completer.use_jedi = False #auto complete

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import dask.dataframe as dd
import gc
import albumentations
from PIL import Image

import torch
import matplotlib.pyplot as plt
%matplotlib inline
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
    

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

!pip install iterative-stratification
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

**create folder**

In [ ]:
if __name__ == "__main__":
  df = pd.read_csv("../input/bengaliai-cv19/train.csv")
  print(df.head())
  df.loc[:, 'kfold'] = -1

  df = df.sample(frac = 1).reset_index(drop=True)

  X = df.image_id.values
  y = df[["grapheme_root", "vowel_diacritic", "consonant_diacritic"]].values

  mskf = MultilabelStratifiedKFold(n_splits= 5)

  for folder, (train_, validation_) in enumerate(mskf.split(X, y)):
    print("Train: ", train_, "Validation: ", validation_)
    df.loc[validation_, 'kfold'] = folder
  
  print(df.kfold.value_counts())
  df.to_csv("./train_folder.csv", index = False)

del df 
del mskf
del X
del y
gc.collect()

**check dataframes**

In [ ]:
# df1 = pd.read_parquet("../input/bengaliai-cv19/train_image_data_0.parquet")
# df1.head()

**create image pickels** 
(to read images)

In [ ]:
import joblib
import glob #list files
from tqdm import tqdm

In [ ]:
!mkdir image_pickels

In [ ]:
# from dask.distributed import Client
# client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='16GB')
# client

In [ ]:
# df=0
# image_ids = 0
# image_array = 0
# if __name__ == "__main__":
#     files = glob.glob("../input/bengaliai-cv19/train_*.parquet")
#     for f in files:
#         del df 
#         del image_ids
#         del image_array
#         gc.collect()
#         df = pd.read_parquet(f)
#         image_ids = df.image_id.values
#         df = df.drop("image_id", axis = 1)
#         image_array = df.values
#         for j, img_id in tqdm(enumerate(image_ids), total = len(image_ids)):
#             joblib.dump(image_array[j, :], f"./image_pickels/{img_id}.pkl")

In [ ]:
df=0
image_ids = 0
image_array = 0
if __name__ == "__main__":
    df = pd.read_parquet("../input/bengaliai-cv19/train_image_data_0.parquet")
    image_ids = df.image_id.values
    df = df.drop("image_id", axis = 1)
    image_array = df.values
    for j, img_id in tqdm(enumerate(image_ids), total = len(image_ids)):
        joblib.dump(image_array[j, :], f"./image_pickels/{img_id}.pkl")
del df 
del image_ids
del image_array
gc.collect()

In [ ]:
df=0
image_ids = 0
image_array = 0
if __name__ == "__main__":
    df = pd.read_parquet("../input/bengaliai-cv19/train_image_data_1.parquet")
    image_ids = df.image_id.values
    df = df.drop("image_id", axis = 1)
    image_array = df.values
    for j, img_id in tqdm(enumerate(image_ids), total = len(image_ids)):
        joblib.dump(image_array[j, :], f"./image_pickels/{img_id}.pkl")
del df 
del image_ids
del image_array
gc.collect()

In [ ]:
df=0
image_ids = 0
image_array = 0
if __name__ == "__main__":
    df = pd.read_parquet("../input/bengaliai-cv19/train_image_data_2.parquet")
    image_ids = df.image_id.values
    df = df.drop("image_id", axis = 1)
    image_array = df.values
    for j, img_id in tqdm(enumerate(image_ids), total = len(image_ids)):
        joblib.dump(image_array[j, :], f"./image_pickels/{img_id}.pkl")
del df 
del image_ids
del image_array
gc.collect()

In [ ]:
df=0
image_ids = 0
image_array = 0
if __name__ == "__main__":
    df = pd.read_parquet("../input/bengaliai-cv19/train_image_data_3.parquet")
    image_ids = df.image_id.values
    df = df.drop("image_id", axis = 1)
    image_array = df.values
    for j, img_id in tqdm(enumerate(image_ids), total = len(image_ids)):
        joblib.dump(image_array[j, :], f"./image_pickels/{img_id}.pkl")
del df 
del image_ids
del image_array
gc.collect()

In [ ]:
del df 
del image_ids
del image_array
gc.collect()

**dataset check/view**

In [ ]:
class TrainD:
    def __init__(self, folder, img_height, img_width, mean, std ):
        df = pd.read_csv("./train_folder.csv")
        y = df[["image_id", "grapheme_root", "vowel_diacritic", "consonant_diacritic", "kfold"]]
        df = df[df.kfold.isin(folder)].reset_index(drop = True)
        
        self.image_ids = df.image_id.values
        self.grapheme_root = df.grapheme_root.values        
        self.vowel_diacritic = df.vowel_diacritic.values        
        self.consonant_diacritic = df.consonant_diacritic.values   
        
        if len(folder) == 1:
            self.aug = albumentations.Compose([
                albumentations.Resize(img_height, img_width, always_apply = True),
                albumentations.Normalize(mean, std, always_apply = True)
            ])
        else:         
            self.aug = albumentations.Compose([
                albumentations.Resize(img_height, img_width, always_apply = True),
                albumentations.ShiftScaleRotate(shift_limit=0.0625, scale_limit=.01, rotate_limit=5, p=0.9),
                albumentations.Normalize(mean, std, always_apply = True)
            ])
        
    def __len__(self):
        return len(self.image_ids)
    def __getitem__(self, item):
        image = joblib.load(f"./image_pickels/{self.image_ids[item]}.pkl")
        image = image.reshape(137,236).astype(float)
        image = Image.fromarray(image).convert("RGB")
        image = self.aug(image= np.array(image))["image"]
        image = np.transpose(image, (2,0,1)).astype(np.float32)
        return {
            'image': torch.tensor(image, dtype = torch.float),
            'grapheme_root': torch.tensor(self.grapheme_root[item], dtype = torch.long),
            'vowel_diacritic': torch.tensor(self.vowel_diacritic[item], dtype = torch.long),
            'consonant_diacritic': torch.tensor(self.consonant_diacritic[item], dtype = torch.long),

        }

In [ ]:
dataset = TrainD(folder=[0,1], img_height = 137, img_width=236, mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))

In [ ]:
idx = 69
img = dataset[idx]["image"]
print(dataset[idx]["grapheme_root"])
print(dataset[idx]["vowel_diacritic"])
print(dataset[idx]["consonant_diacritic"])
npimg = img.numpy()
plt.imshow(np.transpose(npimg, (1,2,0)))

**model**

In [ ]:
import sys
pt= "../input/pretrained-models/pretrained-models.pytorch-master"
sys.path.insert(0, pt)

In [ ]:
import pretrainedmodels
import torch.nn as nn
from torch.nn import functional as fnc

class ResNet34(nn.Module):
    def __init__(self, pretrained):
        super(ResNet34, self).__init__()
        if pretrained is True:
            self.model = pretrainedmodels.__dict__["resnet34"](pretrained="imagenet")
        else:
            self.model = pretrainedmodels.__dict__["resnet34"](pretrained=None)

        self.l0 = nn.Linear(512,168)
        self.l1 = nn.Linear(512,11)
        self.l2 = nn.Linear(512,7)

    def forward(self, x):
        bs, _,_, _ = x.shape
        x = self.model.features(x)
        x = fnc.adaptive_avg_pool2d(x, 1).reshape(bs, -1)
        l0 = self.l0(x)
        l1 = self.l1(x)
        l2 = self.l2(x)
        return l0, l1, l2


**model dispatcher**

In [ ]:
Model_dispatcher = {
    'resnet34': ResNet34
}

**training**

In [ ]:
#variables
import ast

device = "cuda"
trainFoldCSV = "./train_folder.csv"
imgHeight = 137
imgWidth = 236
epochs = 50
baseModel = 'resnet34'

trainSize = 64 #128/256 
testSize = 8

modelMean = ast.literal_eval("(0.485, 0.456, 0.406)")
modelStd = ast.literal_eval("(0.229, 0.224, 0.225)")

trainFolder = ast.literal_eval("(0, 1, 2, 3)")
validFolder = ast.literal_eval("(4,)")
#testFolder




In [ ]:
def train(dataset, data_loader, model, optimizer):
    model.train()

    for bi, d in tqdm(enumerate(data_loader), total=int(len(dataset)/data_loader.batch_size)):
        image = d["image"]
        grapheme_root = d["grapheme_root"]
        vowel_diacritic = d["vowel_diacritic"]
        consonant_diacritic = d["consonant_diacritic"]

        image = image.to(device, dtype = torch.float)
        grapheme_root = grapheme_root.to(device, dtype = torch.long)
        vowel_diacritic = vowel_diacritic.to(device, dtype = torch.long)
        consonant_diacritic = consonant_diacritic.to(device, dtype = torch.long)

        optimizer.zero_grad()
        outputs = model(image)
        targets = (grapheme_root, vowel_diacritic, consonant_diacritic)
        loss = loss_fn(outputs, targets)

        loss.backward()
        optimizer.step()



def evaluate(dataset, data_loader, model):
    model.eval()
    final_loss = 0
    counter = 0
    
    for bi, d in tqdm(enumerate(data_loader), total=int(len(dataset)/data_loader.batch_size)):
        counter = counter + 1
        image = d["image"]
        grapheme_root = d["grapheme_root"]
        vowel_diacritic = d["vowel_diacritic"]
        consonant_diacritic = d["consonant_diacritic"]

        image = image.to(device, dtype = torch.float)
        grapheme_root = grapheme_root.to(device, dtype = torch.long)
        vowel_diacritic = vowel_diacritic.to(device, dtype = torch.long)
        consonant_diacritic = consonant_diacritic.to(device, dtype = torch.long)

        outputs = model(image)
        targets = (grapheme_root, vowel_diacritic, consonant_diacritic)
        loss = loss_fn(outputs, targets)
        final_loss = final_loss + loss
    return final_loss / counter

def loss_fn(outputs, targets):
    o1, o2, o3 = outputs
    t1, t2, t3 = targets
    l1 = nn.CrossEntropyLoss()(o1, t1)
    l2 = nn.CrossEntropyLoss()(o2, t2)
    l3 = nn.CrossEntropyLoss()(o3, t3)
    return (l1+ l2+ l3) / 3


def main():
    model = ResNet34(pretrained = True)
    model.to(device)
    
    

    train_dataset = TrainD(
        folder =trainFolder,
        img_height = imgHeight,
        img_width = imgWidth,
        mean= modelMean,
        std = modelStd
    )

    train_loader = torch.utils.data.DataLoader(
        dataset = train_dataset,
        batch_size = trainSize,
        shuffle = True,
        num_workers = 4
    )

    valid_dataset = TrainD(
        folder =validFolder,
        img_height = imgHeight,
        img_width = imgWidth,
        mean= modelMean,
        std = modelStd
    )

    valid_loader = torch.utils.data.DataLoader(
        dataset = valid_dataset,
        batch_size = testSize,
        shuffle = False,
        num_workers = 4
    )

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode = "min", patience = 5, factor = 0.3, verbose = True)

#     if torch.cuda.device_count() >1:
#         model = nn.DataParallel(model)
    #earlystopping
    for epoch in range(epochs):
        train(train_dataset, train_loader, model, optimizer)
        val_score = evaluate(valid_dataset, valid_loader, model)
        scheduler.step(val_score)
        torch.save(model.state_dict(), f"{baseModel}_fold{validation_folder[0]}.bin")


main() 

In [ ]:
trainFolder = ast.literal_eval("(0, 1, 2, 4)")
validFolder = ast.literal_eval("(3,)")
#testFolder
main()

In [ ]:
trainFolder = ast.literal_eval("(0, 1, 3, 4)")
validFolder = ast.literal_eval("(2)")
#testFolder
main()

In [ ]:
trainFolder = ast.literal_eval("(0, 2, 3, 4)")
validFolder = ast.literal_eval("(1,)")
#testFolder
main()

In [ ]:
trainFolder = ast.literal_eval("(1, 2, 3, 4)")
validFolder = ast.literal_eval("(0,)")
#testFolder
main()

**test set**

In [ ]:
testSize = 32
class TestD:
    def __init__(self, df, img_height, img_width, mean, std ):
#         df = pd.read_csv("./train_folder.csv")
#         y = df[["image_id", "grapheme_root", "vowel_diacritic", "consonant_diacritic", "kfold"]]
#         df = df[df.kfold.isin(folder)].reset_index(drop = True)
        
        self.image_ids = df.image_id.values
        self.img_arr = df.iloc[:, 1:].values
        
#         self.grapheme_root = df.grapheme_root.values        
#         self.vowel_diacritic = df.vowel_diacritic.values        
#         self.consonant_diacritic = df.consonant_diacritic.values   
        
        self.aug = albumentations.Compose([
            albumentations.Resize(img_height, img_width, always_apply = True),
            albumentations.Normalize(mean, std, always_apply = True)
        ])

        
    def __len__(self):
        return len(self.image_ids)
    def __getitem__(self, item):
        image = self.img_arr[item, :]
        img_id = self.image_ids[item]
        image = image.reshape(137,236).astype(float)
        image = Image.fromarray(image).convert("RGB")
        image = self.aug(image= np.array(image))["image"]
        image = np.transpose(image, (2,0,1)).astype(np.float32)
        return {
            'image': torch.tensor(image, dtype = torch.float),
            'image_id': img_id

        }